# <h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>SUDOKU PUZZLE </center></h1> 

Solving a sudoku puzzle from an unsolved image

The project consists of three parts :

**Part One : Digit Classification Model**

**Part Two : Detection and Reading Sudoku from an image**
    
**Part Three : Solving the puzzle**



<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>TABLE OF CONTENTS</center></h1>

[1. IMPORTING LIBRARIES](#1)
    
[2. PART ONE: DIGIT CLASSIFICATION MODEL](#2)    

[3. LOADING DATA](#3)        

[4. SPLITTING DATASET](#4) 
    
[5. DATA PREPROCESSING](#5)

[6. MODEL BUILDING](#6) 

[7. PART TWO: READING THE SUDOKU PUZZLE](#7) 
    
[8. DETECTING CONTOUR](#8)    

[9. SPLITTING THE CELLS AND CLASSIFYING DIGITS](#9)        

[10. PART THREE: SOLVING THE SUDOKU](#10) 

[11. FIN](#11) 

 <a id="1"></a>
# **<span style="color:#4686C8;">IMPORTING LIBRARIES</span>**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import cv2
from glob import glob

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from pathlib import Path
from PIL import Image



 <h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>PART ONE</center></h1> 

<a id="2"></a>
# **<span style="color:#4686C8;">DIGIT CLASSIFICATION MODEL</span>** 

**In this section :**
- Loading data
- Splitting data into training, testing and validation sets
- Preprocessing the data
- Model building and training

<a id="3"></a>
# **<span style="color:#4686C8;">LOADING DATA</span>** 

In [ ]:
# Loading the data

data = os.listdir(r"../input/digits/Digits")
data_X = []
data_y = []
data_classes = len(data)

for i in range(0, data_classes):
    data_list = os.listdir(r"../input/digits/Digits" +"/"+str(i))
    for j in data_list:
        pic = cv2.imread(r"../input/digits/Digits" +"/"+str(i)+"/"+j)
        pic = cv2.resize(pic, (32,32))
        data_X.append(pic)
        data_y.append(i)
        
if(len(data_X) == len(data_y)):
    print("Total datapoints : ", len(data_X))
    
data_X = np.array(data_X)
data_y = np.array(data_y)

<a id="4"></a>
# **<span style="color:#4686C8;">SPLITTING DATASET</span>** 

In [ ]:
#Spliting the train validation and test sets

train_X, test_X, train_y, test_y = train_test_split(data_X,data_y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)
print("Training Set Shape = ",train_X.shape)
print("Validation Set Shape = ",valid_X.shape)
print("Test Set Shape = ",test_X.shape)

<a id="5"></a>
# **<span style="color:#4686C8;">DATA PREPROCESSING</span>** 

In [ ]:
# Preprocessing the images for neuralnet

def Prep(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #making image grayscale
    img = cv2.equalizeHist(img) #Histogram equalization to enhance contrast
    img = img/255 #normalizing
    return img

train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))

#Reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

#Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

In [ ]:
# One hot encoding

train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

<a id="6"></a>
# **<span style="color:#4686C8;">MODEL BUILDING</span>** 

**Building a Convolutional Neural Network :**
- Initialising the Convnet
- Defining by adding layers
- Compile the Convnet
- Train the Convnet

In [ ]:
# Build the model

model = Sequential([
    Conv2D(60, (5,5), activation = 'relu', padding = 'same', input_shape = (32,32,1)),
    Conv2D(60, (5,5), activation = 'relu', padding=  'same'),
    MaxPooling2D(pool_size = (2,2)),
    
    Conv2D(30, (5,5), activation = 'relu', padding = 'same'),
    Conv2D(30, (5,5), activation = 'relu', padding = 'same'),
    MaxPooling2D(pool_size = (2,2), strides = (2,2)),
    Dropout(0.50),
    
    Flatten(),
    Dense(500, activation = 'relu'),
    Dropout(0.5),
    Dense(10, activation = 'softmax')
])

In [ ]:
# Check summary of the model
model.summary()

In [ ]:
# Compile the model

optimizer = RMSprop(learning_rate = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0)
model.compile(loss = 'categorical_crossentropy',
             optimizer = optimizer,
             metrics = ['accuracy'])

In [ ]:
# Fit the model

history = model.fit(datagen.flow(train_X, train_y, batch_size = 32),
                   epochs = 30,
                   validation_data = (valid_X, valid_y),
                   verbose = 2,
                   steps_per_epoch = 200)

In [ ]:
# Testing the model on the test set

score = model.evaluate(test_X, test_y, verbose = 0)
print("Test Score : ", score[0])
print("Test Accuracy : ", score[1])

# <h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>PART TWO</center></h1> 

<a id="7"></a>
# **<span style="color:#4686C8;">READING THE SUDOKU PUZZLE </span>**

**In this section :**
- Read an image from the dataset
- Preprocess the image

In [ ]:
# Randomly select an image from the dataset

folder=r"../input/sudoku-box-detection/aug"

a = random.choice(os.listdir(folder))
print(a)
sudoku_a = cv2.imread(folder + '/' + a)
plt.figure()
plt.imshow(sudoku_a)
plt.show()

In [ ]:
#Preprocessing image to be read
sudoku_a = cv2.resize(sudoku_a, (450,450))

# function to greyscale, blur and change the receptive threshold of image
def preprocess(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (3,3),6) 
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    return threshold_img

threshold = preprocess(sudoku_a)

In [ ]:
# Let's look at what we have got
plt.figure()
plt.imshow(threshold)
plt.show()

<a id="8"></a>
# **<span style="color:#4686C8;">DETECTING CONTOUR</span>**

- Detect the biggest contour of the image
- Reshaping the outline to get the cropped and well-aligned sudoku

In [ ]:
# Finding the outline of the puzzle in the sudoku image
contour_1 = sudoku_a.copy()
contour_2 = sudoku_a.copy()
contour, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(contour_1, contour, -1, (0,255,0), 3)

# Let's see what we got
plt.figure()
plt.imshow(contour_1)
plt.show()

In [ ]:
def main_outline(contour):
    biggest = np.array([])
    max_area = 0
    for i in contour:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest, max_area

def reframe(points):
    points = points.reshape((4,2))
    points_new = np.zeros((4,1,2), dtype = np.int32)
    add = points.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] = points[np.argmax(add)]
    diff = np.diff(points, axis = 1)
    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]
    return points_new

def splitcells(img):
    rows = np.vsplit(img, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

black_img = np.zeros((450,450,3), np.uint8)
biggest, maxArea = main_outline(contour)
if biggest.size != 0:
    biggest = reframe(biggest)
    cv2.drawContours(contour_2, biggest, -1, (0,255,0), 10)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[450,0],[0,450],[450,450]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imagewrap = cv2.warpPerspective(sudoku_a, matrix, (450,450))
    imagewrap = cv2.cvtColor(imagewrap, cv2.COLOR_BGR2GRAY)
    
plt.figure()
plt.imshow(imagewrap)
plt.show()

In [ ]:
# Import puzzle to be solved
puzzle = cv2.imread("../input/su-puzzle/su.jpg")

# Let's see what we got
plt.figure()
plt.imshow(puzzle)
plt.show()

In [ ]:
# Resizing the puzzle to be solved
puzzle = cv2.resize(puzzle, (450, 450))

# Preprocessing puzzle
su_puzzle = preprocess(puzzle)

# Finding the outline of the sudoku puzzle in the image
su_contour_1 = su_puzzle.copy()
su_contour_2 = sudoku_a.copy()
su_contour, hierarchy = cv2.findContours(su_puzzle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(su_contour_1, su_contour, -1, (0,255,0), 3)

black_img = np.zeros((450,450,3), np.uint8)
su_biggest, su_maxArea = main_outline(su_contour)
if su_biggest.size != 0:
    su_biggest = reframe(su_biggest)
    cv2.drawContours(su_contour_2,su_biggest,-1, (0,255,0),10)
    su_pts1 = np.float32(su_biggest)
    su_pts2 = np.float32([[0,0],[450,0],[0,450],[450,450]])
    su_matrix = cv2.getPerspectiveTransform(su_pts1,su_pts2)  
    su_imagewrap = cv2.warpPerspective(puzzle,su_matrix,(450,450))
    su_imagewrap =cv2.cvtColor(su_imagewrap, cv2.COLOR_BGR2GRAY)
    
plt.figure()
plt.imshow(su_imagewrap)
plt.show()

<a id="9"></a>
# **<span style="color:#4686C8;">SPLITTING THE CELLS AND CLASSIFYING DIGITS</span>**

- Splitting the sudoku box in 81 cell with empty spaces or digits
- Cropping the cells to avoid misdetection of boundary lines as digits
- Using the model to classify the digits in the cells such that the empty cells are classified as zero
- Getting the detected output in the form of an array of 81 digits

In [ ]:
sudoku_cell = splitcells(su_imagewrap)

# Let's have a look at the last cell
plt.figure()
plt.imshow(sudoku_cell[58])
plt.show()

In [ ]:
# The sudoku cell's output contains the boundaries which could lead to misclassifications by the model
# Cropping the cells to avoid that

def CropCell(cells):
    cells_cropped = []
    for image in cells:
        img = np.array(image)
        img = img[4:46, 6:46]
        img = Image.fromarray(img)
        cells_cropped.append(img)
    return cells_cropped

sudoku_cell_cropped = CropCell(sudoku_cell)

# Let's have a look at the last cell
plt.figure()
plt.imshow(sudoku_cell_cropped[58])
plt.show()

In [ ]:
def read_cells(cell, model):
    result = []
    for image in cell:
        # Preprocess the image as it was in the model
        img = np.asarray(image)
        img = img[4:img.shape[0] - 4, 4:img.shape[1] - 4]
        img = cv2.resize(img, (32,32))
        img = img/255
        img = img.reshape(1,32,32,1)
        
        # Getting the predictions and setting the values if the probabilites are above 65%
        predictions = model.predict(img)
        classIndex=  np.argmax(predictions, axis = 1)
        probabilityValue = np.amax(predictions)
        
        if(probabilityValue > 0.65):
            result.append(classIndex[0])
        else:
            result.append(0)
    return result

grid = read_cells(sudoku_cell_cropped, model)
grid = np.asarray(grid)

# <h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>PART THREE</center></h1> 

<a id="9"></a>
# **<span style="color:#4686C8;">SOLVING THE SUDOKU</span>**

- Reshaping the array into a 9x9 matrix
- Solving the matrix using recursion

In [ ]:
# Reshaping the array into a 9x9 matrix
grid = np.reshape(grid, (9,9))
grid

In [ ]:
# For comparison
plt.figure()
plt.imshow(su_imagewrap)
plt.show()

In [ ]:
# This function finds the next box to solve
def next_box(quiz):
    for row in range(9):
        for col in range(9):
            if(quiz[row][col] == 0):
                return (row, col)
    return False

# Function to fill the possible values by evaluating rows columns and smaller cells
def possible(quiz, row, col, n):
    # global quiz
    for i in range(0,9):
        if(quiz[row][i] == n and row != i):
            return False
    for i in range(0,9):
        if(quiz[i][col] == n and col != i):
            return False
    row0 = (row)//3
    col0 = (col)//3
    for i in range(row0*3, row0*3 + 3):
        for j in range(col0*3, col0*3 + 3):
            if(quiz[i][j] == n and (i,j) != (row,col)):
                return False
    return True


# Recursive function to loop over until a valid answer is found
def solve(quiz):
    val = next_box(quiz)
    if val is False:
        return True
    else : 
        row, col = val
        for n in range(1,10): #n is the possible solution
            if possible(quiz,row, col, n):
                quiz[row][col]=n
                if solve(quiz):
                    return True 
                else:
                    quiz[row][col]=0
        return 

    
def Solved(quiz):
    for row in range(9):
        if row % 3 == 0 and row != 0:
            print("....................")

        for col in range(9):
            if col % 3 == 0 and col != 0:
                print("|", end=" ")

            if col == 8:
                print(quiz[row][col])
            else:
                print(str(quiz[row][col]) + " ", end="")

In [ ]:
solve(grid)

In [ ]:
if solve(grid):
    Solved(grid)
else:
    print("Solution don't exist. Model misread digits.")

<a id="11"></a>
<h1 style='background:#4686C8; border:0; color:#5CE1E6'><center>FIN</center></h1> 